
<div style="
    background-color: #f7f7f7;
    background-image: url(''), url('') ;
    background-position: left bottom, right top;
    background-repeat: no-repeat,  no-repeat;
    background-size: auto 60px, auto 160px;
    border-radius: 5px;
    box-shadow: 0px 3px 1px -2px rgba(0, 0, 0, 0.2), 0px 2px 2px 0px rgba(0, 0, 0, 0.14), 0px 1px 5px 0px rgba(0,0,0,.12);">

<h1 style="
    color: #2a4cdf;
    font-style: normal;
    font-size: 2.25rem;
    line-height: 1.4em;
    font-weight: 600;
    padding: 30px 200px 0px 30px;"> 
        Getting started with Baysian Optimization for automated workflows</h1>
<p style="font-size: 1.25em; font-style: italic; padding: 5px 200px 30px 30px;">
    Lauri Himanen</p>
</div>

# Introduction

This tutorial will help you get started with using Baysian Optimation for automating the sampling of experiments/simulations. The main goal is this: you want to optimize certain parameters in your data (e.g. yield, efficiency, energy), but because there are several input parameters that control the optimization target, it is hard to guide this optimization manually or through analytic reasoning. This is where Baysian Optimization can help.


# Install dependencies

In [ ]:
!pip install baybe

# Define the search space

In [ ]:
from baybe.parameters import (
    CategoricalParameter,
    NumericalDiscreteParameter,
    SubstanceParameter,
)

parameters = [
    CategoricalParameter(
        name="Granularity",
        values=["coarse", "medium", "fine"],
        encoding="OHE",  # one-hot encoding of categories
    ),
    NumericalDiscreteParameter(
        name="Pressure[bar]",
        values=[1, 5, 10],
        tolerance=0.2,  # allows experimental inaccuracies up to 0.2 when reading values
    ),
    SubstanceParameter(
        name="Solvent",
        data={
            "Solvent A": "COC",
            "Solvent B": "CCC",  # label-SMILES pairs
            "Solvent C": "O",
            "Solvent D": "CS(=O)C",
        },
        encoding="MORDRED",  # chemical encoding via mordred package
    ),
]

# Defining the optimization objective


In [ ]:
from baybe.targets import NumericalTarget
from baybe.objectives import SingleTargetObjective

target = NumericalTarget(
    name="Yield",
    mode="MAX",
)
objective = SingleTargetObjective(target=target)

# Define acquisition strategy

In [ ]:
from baybe.recommenders import (
    BotorchRecommender,
    FPSRecommender,
    TwoPhaseMetaRecommender,
)

recommender = TwoPhaseMetaRecommender(
    initial_recommender=FPSRecommender(),  # farthest point sampling
    recommender=BotorchRecommender(),  # Bayesian model-based optimization
)

# The optimization loop

In [ ]:
from baybe import Campaign

campaign = Campaign(searchspace, objective, recommender)

## Get initial recommendations

In [ ]:
df = campaign.recommend(batch_size=3)
print(df)

In [ ]:
def create_entry():
    """In this function you can decide how the actual experiment/simulation is performed. There are several alternatives:
	- Maybe you can control measurement devices directly through API calls
	- Maybe you wait until someone manually inserts the experiment results into NOMAD
	- Maybe you run the simulation in this notebook
	- Maybe you run the simulation using an HPC batch system
	- etc.
	""""
	pass

In [ ]:
target = 0.1
result = 0
while(result < target):
    df = campaign.recommend(batch_size=1)
	archive = create_entry()
	result = archive.yield
	campaign.add_measurements(archive.yield)